# <center> Université Paris Saclay M2 Data Science </center>#
# <center> TEXT MINING CHATBOTS </center>#
#  NAMES: PANONGBENE JEAN MOHAMED SAWADOGO #
### Email: amet1900@gmail.com #
### Github: https://github.com/Panongbene #

In [ ]:
import re
import string
! pip install fasttext 
import fasttext
import unicodedata 
from time import time
from nltk.corpus import stopwords
from gensim.models import FastText
from gensim.models import  Word2Vec

In [ ]:
# French Stop Word
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Extract file directly in my drive
#https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
#from google.colab import drive
#drive.mount("/content/drive/", force_remount=True)

#!ls drive/MyDrive/'Colab Notebooks'/data

In [ ]:
# file upload data
with open('drive/MyDrive/Colab Notebooks/data/QUAERO_FrenchPress_traindev.ospl', 'r') as f: 
        QUAERO_FrenchPress = f.readlines()
        
# file upload data
with open('drive/MyDrive/Colab Notebooks/data/QUAERO_FrenchMed_traindev.ospl', 'r') as f: 
        QUAERO_FrenchMed = f.readlines()


In [ ]:
# clean word
def clean_word(data):
    """This function remove all word and caracteres that we dont need"""
    return ''.join(x for x in unicodedata.normalize('NFKD', data) if x in string.ascii_letters).lower() 

In [ ]:
def build_token_word(document):
    """This function take a document and build a list of words"""
    list_values = list()
    for lines in document:
    #words=lines.split(' ')
        sent = re.sub("[^A-Za-z]+", ' ', str(lines))
        words_value=sent.split()
        list_values.append(words_value)
        
    return list_values

In [ ]:
# build model vocabulary
#### Building CBOW wordvectors
def training_cbow(document):
    """Training the cbow model"""
    t = time()
    # initialize
    cbow_model = Word2Vec(min_count=1,window=2,size=100, sg = 0)
    # build model vocabulary

    tokens_list=document
  
    cbow_model.build_vocab(tokens_list)
    # train the model
    cbow_model.train(tokens_list, total_examples=cbow_model.corpus_count, epochs=30, report_delay=1)

    print('Time to build CBOW model vocab: {} mins'.format(round((time() - t) / 60, 2)))
    return cbow_model

In [ ]:
def training_sg(document):
    """Train the skip gram model"""
    t = time()
    # initialize skipgram model
    sg_model = Word2Vec(min_count=1,window=2,size=100, sg = 1,sample=5e-5, alpha=0.05, min_alpha=0.0005 )
    # build model vocabulary
    sg_model.build_vocab(document)

    # train the model
    sg_model.train(document, total_examples=sg_model.corpus_count, epochs=30, report_delay=1)

    print('Time to build Skip gram model vocab: {} mins'.format(round((time() - t) / 60, 2)))
    return sg_model
  

In [ ]:
def stop_word(document):
    """This function take a list of list of word and delete stop word"""
    
    for i in range(len(document)):
        for j in document[i]:
            if (j in stop_words) or (len(j)<=2):
                document[i].remove(j)
    
    return document

In [ ]:
# build tocken
QUAERO_FrenchPress_tocken = stop_word(build_token_word(QUAERO_FrenchPress))
QUAERO_FrenchMed_tocken = stop_word(build_token_word(QUAERO_FrenchMed))

In [ ]:
# Word 2 Vec model
model = Word2Vec(min_count=1,size=100)
model = Word2Vec(min_count=1)
model.build_vocab(QUAERO_FrenchPress_tocken)  # prepare the model vocabulary
model.train(QUAERO_FrenchPress_tocken, total_examples=model.corpus_count, epochs=3)#report_delay=1) 

(2095463, 2320746)

In [ ]:
model.wv.most_similar(["bonjour"])

[('Vannier', 0.9553535580635071),
 ('Rincquesen', 0.9530477523803711),
 ('messieurs', 0.9523866176605225),
 ('Winckler', 0.9522286653518677),
 ('Lecaplain', 0.9405015707015991),
 ('mesdames', 0.9394315481185913),
 ('bienvenue', 0.9348511099815369),
 ('Gomez', 0.9334104657173157),
 ('tentatrices', 0.9315935969352722),
 ('Rey', 0.9287653565406799)]

In [ ]:
# Cbow model
cbow_QUAERO_FrenchPress_tocken = training_cbow(QUAERO_FrenchPress_tocken)
cbow_QUAERO_FrenchMed_tocken = training_cbow(QUAERO_FrenchMed_tocken)

Time to build CBOW model vocab: 0.74 mins
Time to build CBOW model vocab: 0.05 mins


In [ ]:
#  skipgram model
sg_QUAERO_FrenchPress_tocken = training_sg(QUAERO_FrenchPress_tocken)
sg_QUAERO_FrenchMed_tocken = training_sg(QUAERO_FrenchMed_tocken)

Time to build Skip gram model vocab: 0.85 mins
Time to build Skip gram model vocab: 0.06 mins


In [ ]:
# test skip gram model
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["traitement"])

[('geait', 0.6312866806983948),
 ('dicamenteux', 0.6190258264541626),
 ('antidouleur', 0.5728226900100708),
 ('Asilah', 0.5521117448806763),
 ('asservissement', 0.5099465847015381),
 ('dissuasion', 0.5068008303642273),
 ('ad', 0.5029745697975159),
 ('paludisme', 0.5003583431243896),
 ('individualis', 0.4930357336997986),
 ('disponibilit', 0.48680031299591064)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["traitement"])

[('pendant', 0.9951863288879395),
 ('arr', 0.9948455095291138),
 ('TYSABRI', 0.9946781396865845),
 ('peuvent', 0.9945305585861206),
 ('doit', 0.9943689107894897),
 ('e', 0.9942471981048584),
 ('plus', 0.9942370057106018),
 ('mois', 0.9941974878311157),
 ('pouvant', 0.9940338730812073),
 ('Tasmar', 0.9939804673194885)]

In [ ]:
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["patient"])

[('infantilis', 0.7272225022315979),
 ('infractionnistes', 0.6173020601272583),
 ('humble', 0.6091175079345703),
 ('neurologique', 0.6038421988487244),
 ('obtenez', 0.6019484400749207),
 ('surv', 0.5935162901878357),
 ('brouillage', 0.5892027616500854),
 ('signaient', 0.586911678314209),
 ('Tourcoing', 0.5844130516052246),
 ('parasitage', 0.5813284516334534)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["patient"])

[('Montrez', 0.9966121315956116),
 ('possibles', 0.9964876770973206),
 ('montrer', 0.9964632391929626),
 ('notice', 0.9964152574539185),
 ('cette', 0.9963333606719971),
 ('alerte', 0.9962992668151855),
 ('aucun', 0.9962983131408691),
 ('survenue', 0.9962772130966187),
 ('conserviez', 0.9961314797401428),
 ('ces', 0.9960231781005859)]

In [ ]:
sg_QUAERO_FrenchPress_tocken.wv.most_similar(["jaune"])

[('grenadine', 0.6964824199676514),
 ('maillot', 0.6784029603004456),
 ('caleront', 0.6563889384269714),
 ('endossant', 0.6198890209197998),
 ('femelles', 0.5915443897247314),
 ('peignoir', 0.5867910981178284),
 ('choire', 0.5840634703636169),
 ('franchie', 0.5811959505081177),
 ('Pena', 0.5647939443588257),
 ('empar', 0.5572044849395752)]

In [ ]:
sg_QUAERO_FrenchMed_tocken.wv.most_similar(["jaune"])

[('thylcellulose', 0.9980731010437012),
 ('Ethylcellulose', 0.9980403184890747),
 ('orange', 0.9979244470596313),
 ('Oxyde', 0.9978638291358948),
 ('fer', 0.9978249669075012),
 ('dioxyde', 0.9978059530258179),
 ('Jaunisse', 0.9977802634239197),
 ('mosa', 0.9977531433105469),
 ('oxyde', 0.9977342486381531),
 ('Psammechinus', 0.9976934194564819)]

In [ ]:
dimension =100
fasttext_QUAERO_FrenchMed_tocken = FastText(QUAERO_FrenchMed_tocken, size=dimension, window=2, min_count=1, workers=4, sg=0)
fasttext_QUAERO_FrenchPress_tocken = FastText(QUAERO_FrenchPress_tocken, size=dimension, window=2, min_count=1, workers=4, sg=0)

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["traitement"])

[('traitements', 0.9999978542327881),
 ('Traitement', 0.9999977350234985),
 ('Taaitement', 0.999997079372406),
 ('allaitement', 0.9999966621398926),
 ('Allaitement', 0.9999962449073792),
 ('vitement', 0.9999956488609314),
 ('Traitements', 0.9999952912330627),
 ('troitement', 0.999995231628418),
 ('lentement', 0.9999949932098389),
 ('partement', 0.9999945163726807)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["traitement"])

[('vachement', 0.9860249757766724),
 ('chement', 0.9853428602218628),
 ('doucement', 0.9844817519187927),
 ('chichement', 0.9844160079956055),
 ('chauffement', 0.9841684699058533),
 ('troitement', 0.9833940863609314),
 ('chirement', 0.982938826084137),
 ('sagement', 0.9827824234962463),
 ('chouement', 0.9827768802642822),
 ('sauvagement', 0.9823540449142456)]

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["jaune"])

[('antithrombotiques', 0.9999393820762634),
 ('sanctions', 0.999937891960144),
 ('instrumentale', 0.9999378323554993),
 ('mentionn', 0.9999376535415649),
 ('antithrombotique', 0.9999369382858276),
 ('consommation', 0.9999363422393799),
 ('bronchographiques', 0.9999361634254456),
 ('niques', 0.9999361038208008),
 ('cropsiques', 0.9999358654022217),
 ('Actions', 0.9999354481697083)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["jaune"])

[('Neptune', 0.9911396503448486),
 ('lune', 0.9889525771141052),
 ('Jeune', 0.9860552549362183),
 ('brune', 0.9858811497688293),
 ('dune', 0.9845155477523804),
 ('youne', 0.9813197255134583),
 ('lagune', 0.9727632999420166),
 ('aune', 0.9643239974975586),
 ('Tribune', 0.9534322023391724),
 ('doune', 0.952678382396698)]

In [ ]:
fasttext_QUAERO_FrenchMed_tocken.wv.most_similar(["Picasso"])

[('iso', 0.9635076522827148),
 ('guid', 0.9608737230300903),
 ('ANASTOMOSES', 0.9606253504753113),
 ('Typho', 0.9602170586585999),
 ('isol', 0.9601095914840698),
 ('Emil', 0.9599283933639526),
 ('THYROUIDE', 0.9598098993301392),
 ('isolee', 0.9597572088241577),
 ('Cas', 0.9596304893493652),
 ('Bignami', 0.9595298767089844)]

In [ ]:
fasttext_QUAERO_FrenchPress_tocken.wv.most_similar(["Picasso"])

[('Mustansariya', 0.9673168063163757),
 ('Ricardo', 0.9648245573043823),
 ('Essouiri', 0.9638747572898865),
 ('Dassault', 0.9624403715133667),
 ('Istanbul', 0.9621859788894653),
 ('Vassili', 0.9618462324142456),
 ('Ouganda', 0.961167573928833),
 ('Atangana', 0.9604490995407104),
 ('Kassim', 0.960124135017395),
 ('Golinsky', 0.959934651851654)]